In [3]:
import numpy as np
import pandas as pd

In [4]:
#import files
#read the txt file
with open('spa.txt') as f:
  text=f.read()
print(text[:50])

Go.	Ve.
Go.	Vete.
Go.	Vaya.
Go.	Váyase.
Hi.	Hola.



In [5]:
data=text.split('\n')
data=[i.split('\t') for i in data]


In [6]:
df=pd.DataFrame(data,columns=['English','Spanish'])

length=len(data)
df=df.sample(length//2)
df.head()

,English,Spanish
34375,What did the pilot say?,¿Qué dijo el piloto?
90774,"I will ask him about it tomorrow, then.",Entonces mañana le pregunto.
118614,"Even though Tom was brought up in the country,...","A pesar de que Tom fue criado en el campo, le ..."
81704,Who's responsible for this problem?,¿Quién es responsable de este problema?
70137,"Sing the song once more, please.","Por favor, canta la canción otra vez."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59482 entries, 34375 to 25653
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   English  59482 non-null  object
 1   Spanish  59482 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [8]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [9]:
df['English']=df['English'].astype(str)
df['Spanish']=df['Spanish'].astype(str)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59482 entries, 0 to 59481
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   English  59482 non-null  object
 1   Spanish  59482 non-null  object
dtypes: object(2)
memory usage: 929.5+ KB


In [11]:
df['English'] = df['English'].fillna('').apply(lambda x: x.lower())
df['Spanish'] = df['Spanish'].fillna('').apply(lambda x: x.lower())
# df['English'] = df['English'].str.replace('.', '')
# df['Spanish'] = df['Spanish'].str.replace('.', '')
df['English'] = df['English'].str.replace(r'\.', '', regex=True)
df['Spanish'] = df['Spanish'].str.replace(r'\.', '', regex=True)

# Tokenization

In [12]:
!pip install --upgrade spacy
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

import spacy

spacy_en = spacy.load("en_core_web_sm")
spacy_es = spacy.load("es_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 98.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
import spacy

spacy_en = spacy.load("en_core_web_sm")
spacy_es = spacy.load("es_core_news_sm")


In [14]:
def tokenize_en(text):
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]

def tokenize_es(text):
    return [tok.text.lower() for tok in spacy_es.tokenizer(text)]

In [15]:
df['English']=df['English'].apply(tokenize_en)
df['Spanish']=df['Spanish'].apply(tokenize_es)

In [16]:
df.sample(5)

,English,Spanish
57592,"[i, saw, tom, die]","[vi, a, tom, morir]"
30849,"[if, you, stand, on, this, stool, ,, you, can,...","[si, te, levantas, sobre, este, taburete, ,, p..."
9728,"[no, matter, what, you, do, ,, you, must, foll...","[no, importa, como, lo, hagas, ,, pero, debes,..."
9919,"[osaka, is, japan, 's, second, biggest, city]","[osaka, es, la, segunda, ciudad, más, grande, ..."
12621,"[i, did, n't, mean, to, frighten, you]","[no, pretendía, asustarte]"


In [17]:
#apply <start of the sentence> and <end of the sentence> in spanish which is output
df['Spanish'] = df['Spanish'].apply(lambda x: ['<sos>'] + x + ['<eos>'])

In [18]:
df.sample(5)

,English,Spanish
56077,"[we, saw, the, sun, sink, below, the, horizon]","[<sos>, vimos, al, sol, hundirse, bajo, el, ho..."
35410,"[i, 'm, going, to, tell, you, about, a, strang...","[<sos>, les, voy, a, contar, un, caso, curioso..."
22444,"[the, policeman, saved, the, child, from, drow...","[<sos>, el, policía, evitó, que, el, niño, se,..."
18231,"[i, will, tell, her, what, to, say, at, the, m...","[<sos>, le, diré, qué, tiene, que, decir, en, ..."
40844,"[this, tie, and, that, jacket, go, well, toget...","[<sos>, esta, corbata, conjunta, con, aquella,..."


In [19]:
spalen=0
for x in df['Spanish']:
  if len(x)>spalen:
    spalen=len(x)
print(spalen)

englen=0
for x in df['English']:
  if len(x)>englen:
    englen=len(x)
print(englen)

40
37


In [20]:
df['English'] = df['English'].apply(lambda x: x + ['<pad>'] * (englen - len(x)))
df['Spanish'] = df['Spanish'].apply(lambda x: x + ['<pad>'] * (spalen - len(x)))

In [21]:

engword = df['English'].explode().unique()
spaword = df['Spanish'].explode().unique()
print(len(engword))
print(len(spaword))

10392
19210


In [22]:
eng_tok_stoi = dict([(char, i) for i,char in enumerate(engword)])
spa_tok_stoi = dict([(char, i) for i,char in enumerate(spaword)])

eng_tok_itos = dict([(i,char)for i ,char in enumerate(engword)])
spa_tok_itos = dict([(i,char)for i ,char in enumerate(spaword)])

In [23]:
df['English']=df['English'].apply(lambda x: [eng_tok_stoi[word] for word in x])
df['Spanish']=df['Spanish'].apply(lambda x: [spa_tok_stoi[word] for word in x])

In [24]:
print('English vocab:',len(eng_tok_itos))
print('Spanish vocab:',len(spa_tok_itos))

English vocab: 10392
Spanish vocab: 19210


In [25]:
train_len = int(len(df)*0.8)
train_len
#47585 rows of data

47585

In [26]:
X_train = df['English'].iloc[:int(train_len)]
y_train = df['Spanish'].iloc[:int(train_len)]
X_test = df['English'].iloc[int(train_len):]
y_test = df['Spanish'].iloc[int(train_len):]

In [27]:
import torch
from torch.utils.data import Dataset, DataLoader

class TranslationDataset(Dataset):
    def __init__(self, english_data, spanish_data):
        self.english_data = english_data
        self.spanish_data = spanish_data

    def __len__(self):
        return len(self.english_data)

    def __getitem__(self, idx):
        english_seq = torch.tensor(self.english_data.iloc[idx], dtype=torch.long)
        spanish_seq = torch.tensor(self.spanish_data.iloc[idx], dtype=torch.long)
        return english_seq, spanish_seq

# Assuming you have already created your df, englen, spalen, eng_tok_index, spa_tok_index

# Create Dataset instances
train_dataset = TranslationDataset(X_train, y_train)
test_dataset = TranslationDataset(X_test, y_test)

# Create DataLoader instances
batch_size = 64 # You can adjust this
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [28]:
count=0
for english_batch, spanish_batch in train_dataloader:
  count+=1
count
#744*64 ~= 47585


744

In [ ]:
for english_batch, spanish_batch in train_dataloader:
  print("English Batch Shape:", english_batch.shape)
  print("Spanish Batch Shape:", spanish_batch.shape)
  break
#the train_dataloader is of shape (744,64,37) (744,64,40) for english and spanish

English Batch Shape: torch.Size([64, 37])
Spanish Batch Shape: torch.Size([64, 40])


In [32]:
import torch.nn as nn
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, dropout, maxlen=5000):
        super().__init__()
        #torch.arange(0, emb_size, 2) is i=[0, dmodel/2]
        #- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size
        #=>2i/emb_size(d_model) * -log of 10000 #x*loga =loga^x
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])


In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import pandas as pd # For dummy data generation

In [34]:

def tokenize_en_placeholder(text: str):
    return [tok.text.lower() for tok in spacy_en.tokenizer(text)]


def tokenize_es_placeholder(text):
    return [tok.text.lower() for tok in spacy_es.tokenizer(text)]

In [35]:


# --- Transformer Model Components ---

class PositionalEncoding(nn.Module):
    """
    Positional encoding module.
    Adds positional information to the input embeddings.
    Expected input shape: [batch_size, seq_len, d_model]
    """
    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Create a positional encoding matrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # [max_len, 1]
        # Term for calculating frequencies: e^( (0, 2, ..., d_model-2) * (-log(10000)/d_model) )
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Apply sin to even indices and cos to odd indices
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Add a batch dimension: [1, max_len, d_model]
        # This allows broadcasting across the batch
        pe = pe.unsqueeze(0)
        # Register 'pe' as a buffer, so it's part of the model's state
        # but not updated by the optimizer.
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor of shape [batch_size, seq_len, d_model]
        Returns:
            Tensor of shape [batch_size, seq_len, d_model] with positional encodings added.
        """
        # Add positional encoding to the input tensor x.
        # self.pe is [1, max_len, d_model]. We slice it to match x's seq_len.
        # self.pe[:, :x.size(1), :] gives [1, seq_len, d_model]
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


In [36]:

class MultiHeadAttention(nn.Module):
    """
    Multi-head attention mechanism.
    Allows the model to jointly attend to information from different
    representation subspaces at different positions.
    """
    def __init__(self, d_model: int, n_heads: int, dropout: float = 0.1):
        super().__init__()
        # d_model: Total dimension of the model (e.g., 512)
        # n_heads: Number of attention heads (e.g., 8)
        assert d_model % n_heads == 0, "d_model must be divisible by n_heads"

        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads  # Dimension of each head's key/query/value (e.g., 512/8 = 64)

        # Linear layers for Query, Key, Value, and Output
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model) # Output transformation

        self.dropout = nn.Dropout(dropout)

    def scaled_dot_product_attention(self, Q: torch.Tensor, K: torch.Tensor, V: torch.Tensor,
                                     mask: torch.Tensor = None):
        """
        Computes scaled dot-product attention.
        Args:
            Q: Query tensor, shape [batch_size, n_heads, seq_len_q, d_k]
            K: Key tensor, shape [batch_size, n_heads, seq_len_k, d_k]
            V: Value tensor, shape [batch_size, n_heads, seq_len_v, d_k]
            =>sequece_length of q k can be different in decoder
            q is created from the target that is querying and
            k and v is created from the source that is being searched.
            mask: Optional mask tensor, shape broadcastable to [batch_size, n_heads, seq_len_q, seq_len_k]
                  Masked positions should be True or 1.
        Returns:
            context: Output tensor after attention, shape [batch_size, n_heads, seq_len_q, d_k]
            attention_weights: Attention weights, shape [batch_size, n_heads, seq_len_q, seq_len_k]
        """
        # MatMul(Q, K.transpose) / sqrt(d_k)
        # K.transpose(-2, -1) swaps the last two dimensions (seq_len_k and d_k)
        # Note : Matrix multiplication is used in last two layer only
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k) # [B, H, Lq, Lk]

        # Apply mask if provided (e.g., for padding or look-ahead)
        if mask is not None:
            # Masked positions (where mask is True, or 0 if mask indicates valid positions)
            # Assuming mask indicates valid positions with 1/True, invalid with 0/False.
            scores = scores.masked_fill(mask == 0, -1e9) # Fill where mask is False

        # Apply softmax to get attention weights
        attention_weights = F.softmax(scores, dim=-1) # [B, H, Lq, Lk]
        attention_weights = self.dropout(attention_weights)

        # MatMul(attention_weights, V)
        context = torch.matmul(attention_weights, V) # [B, H, Lq, d_k]

        return context, attention_weights

    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor,
                mask: torch.Tensor = None) -> torch.Tensor:
        """
        Forward pass for MultiHeadAttention.
        Args:
            query: Query tensor, shape [batch_size, seq_len_q, d_model]
            key: Key tensor, shape [batch_size, seq_len_k, d_model]
            value: Value tensor, shape [batch_size, seq_len_v, d_model] (seq_len_k == seq_len_v)
            mask: Optional mask, shape broadcastable to [batch_size, 1, seq_len_q, seq_len_k]
        Returns:
            output: Output tensor, shape [batch_size, seq_len_q, d_model]
        """
        batch_size = query.size(0)

        # 1. Linear transformations and reshape into multiple heads
        # query, key, value shapes: [batch_size, seq_len, d_model]
        # After W_q, W_k, W_v: still [batch_size, seq_len, d_model]
        # .view: [batch_size, seq_len, n_heads, d_k]
        # .transpose(1, 2): [batch_size, n_heads, seq_len, d_k]
        Q = self.W_q(query).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        # 2. Apply scaled dot-product attention
        # The mask for scaled_dot_product_attention should be [B, H, Lq, Lk] or broadcastable.
        # If input mask is [B, 1, Lq, Lk] or [B, 1, 1, Lk], it will broadcast.
        attention_context, _ = self.scaled_dot_product_attention(Q, K, V, mask) # attention_context: [B, H, Lq, d_k]

        # 3. Concatenate heads and apply final linear layer
        # .transpose(1, 2): [batch_size, seq_len_q, n_heads, d_k]
        # .contiguous().view: [batch_size, seq_len_q, d_model (n_heads * d_k)]
        attention_context = attention_context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # Final linear transformation
        output = self.W_o(attention_context) # [batch_size, seq_len_q, d_model]

        return output


In [37]:

class FeedForward(nn.Module):
    """
    Position-wise feed-forward network.
    Applied to each position separately and identically.
    """
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        # d_model: Input and output dimension (e.g., 512)
        # d_ff: Inner dimension of the FFN (e.g., 2048)
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x shape: [batch_size, seq_len, d_model]
        # FFN(x) = max(0, xW1 + b1)W2 + b2
        # Here, using ReLU as activation
        return self.linear2(self.dropout(F.relu(self.linear1(x))))


In [38]:

class EncoderLayer(nn.Module):
    """
    Single encoder layer, consisting of self-attention and a feed-forward network.
    """
    def __init__(self, d_model: int, n_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, n_heads, dropout)
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model) # Layer normalization for self-attention
        self.norm2 = nn.LayerNorm(d_model) # Layer normalization for feed-forward
        self.dropout = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor, src_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Args:
            src: Input to the encoder layer, shape [batch_size, src_len, d_model]
            src_mask: Mask for source sequence (e.g., padding mask),
                      shape broadcastable to [batch_size, n_heads, src_len, src_len]
        Returns:
            Output tensor, shape [batch_size, src_len, d_model]
        """
        # 1. Self-attention sublayer
        # Q, K, V are all 'src'
        attn_output = self.self_attention(src, src, src, src_mask)
        # Residual connection and layer normalization
        src = self.norm1(src + self.dropout(attn_output))

        # 2. Feed-forward sublayer
        ff_output = self.feed_forward(src)
        # Residual connection and layer normalization
        src = self.norm2(src + self.dropout(ff_output))

        return src


In [39]:

class DecoderLayer(nn.Module):
    """
    Single decoder layer, consisting of self-attention, cross-attention (encoder-decoder attention),
    and a feed-forward network.
    """
    def __init__(self, d_model: int, n_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.self_attention = MultiHeadAttention(d_model, n_heads, dropout) # Masked self-attention
        self.cross_attention = MultiHeadAttention(d_model, n_heads, dropout) # Encoder-decoder attention
        self.feed_forward = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model) # For self-attention
        self.norm2 = nn.LayerNorm(d_model) # For cross-attention
        self.norm3 = nn.LayerNorm(d_model) # For feed-forward
        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt: torch.Tensor, encoder_output: torch.Tensor,
                src_mask: torch.Tensor = None, tgt_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Args:
            tgt: Input to the decoder layer (target sequence embeddings), shape [batch_size, tgt_len, d_model]
            encoder_output: Output from the encoder, shape [batch_size, src_len, d_model]
            src_mask: Mask for source sequence (padding mask from encoder),
                      shape broadcastable to [batch_size, n_heads, tgt_len (query), src_len (key)]
            tgt_mask: Mask for target sequence (e.g., combined look-ahead and padding mask),
                      shape broadcastable to [batch_size, n_heads, tgt_len, tgt_len]
        Returns:
            Output tensor, shape [batch_size, tgt_len, d_model]
        """
        # 1. Masked Self-Attention sublayer (on target sequence)
        # Q, K, V are all 'tgt'
        self_attn_output = self.self_attention(tgt, tgt, tgt, tgt_mask)
        # Residual connection and layer normalization
        tgt = self.norm1(tgt + self.dropout(self_attn_output))

        # 2. Cross-Attention sublayer (attending to encoder output)
        # Query is 'tgt' (decoder state), Key and Value are 'encoder_output'
        cross_attn_output = self.cross_attention(tgt, encoder_output, encoder_output, src_mask)
        # Residual connection and layer normalization
        tgt = self.norm2(tgt + self.dropout(cross_attn_output))

        # 3. Feed-Forward sublayer
        ff_output = self.feed_forward(tgt)
        # Residual connection and layer normalization
        tgt = self.norm3(tgt + self.dropout(ff_output))

        return tgt


In [40]:

class TransformerEncoder(nn.Module):
    """
    Complete Transformer encoder, stacking multiple EncoderLayers.
    """
    def __init__(self, src_vocab_size: int, d_model: int, n_heads: int,
                 n_layers: int, d_ff: int, max_len: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(src_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        self.dropout = nn.Dropout(dropout) # Dropout for the sum of embeddings and pos encoding

    def forward(self, src: torch.Tensor, src_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Args:
            src: Source sequence tensor (token indices), shape [batch_size, src_len]
            src_mask: Mask for source sequence.
        Returns:
            Output tensor from the encoder, shape [batch_size, src_len, d_model]
        """
        # src shape: [batch_size, src_len]
        # Embedding: [batch_size, src_len, d_model]
        # Scale embedding by sqrt(d_model) as per "Attention is All You Need"
        embedded_src = self.embedding(src) * math.sqrt(self.d_model)
        # Add positional encoding
        pos_encoded_src = self.pos_encoding(embedded_src)
        # Dropout is applied by PositionalEncoding, or can be applied here too if not in PE.

        # Pass through each encoder layer
        encoder_output = pos_encoded_src
        for layer in self.layers:
            encoder_output = layer(encoder_output, src_mask)

        return encoder_output


In [41]:

class TransformerDecoder(nn.Module):
    """
    Complete Transformer decoder, stacking multiple DecoderLayers.
    """
    def __init__(self, tgt_vocab_size: int, d_model: int, n_heads: int,
                 n_layers: int, d_ff: int, max_len: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len, dropout)
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        self.dropout = nn.Dropout(dropout) # Dropout for the sum of embeddings and pos encoding
        self.output_projection = nn.Linear(d_model, tgt_vocab_size) # To get final logits

    def forward(self, tgt: torch.Tensor, encoder_output: torch.Tensor,
                src_mask: torch.Tensor = None, tgt_mask: torch.Tensor = None) -> torch.Tensor:
        """
        Args:
            tgt: Target sequence tensor (token indices), shape [batch_size, tgt_len]
            encoder_output: Output from the encoder, shape [batch_size, src_len, d_model]
            src_mask: Mask for source sequence (for cross-attention).
            tgt_mask: Mask for target sequence (for self-attention).
        Returns:
            Output logits tensor, shape [batch_size, tgt_len, tgt_vocab_size]
        """
        # tgt shape: [batch_size, tgt_len]
        # Embedding: [batch_size, tgt_len, d_model]
        embedded_tgt = self.embedding(tgt) * math.sqrt(self.d_model)
        # Add positional encoding
        pos_encoded_tgt = self.pos_encoding(embedded_tgt)
        # Dropout is applied by PositionalEncoding.

        # Pass through each decoder layer
        decoder_output = pos_encoded_tgt
        for layer in self.layers:
            decoder_output = layer(decoder_output, encoder_output, src_mask, tgt_mask)

        # Final linear layer to project to vocabulary size
        output_logits = self.output_projection(decoder_output)

        return output_logits


In [42]:

class Transformer(nn.Module):
    """
    Complete Transformer model combining Encoder and Decoder.
    """
    def __init__(self, src_vocab_size: int, tgt_vocab_size: int, d_model: int = 512,
                 n_heads: int = 8, n_encoder_layers: int = 6, n_decoder_layers: int = 6,
                 d_ff: int = 2048, max_len: int = 100, dropout: float = 0.1):
        super().__init__()
        self.encoder = TransformerEncoder(
            src_vocab_size, d_model, n_heads, n_encoder_layers, d_ff, max_len, dropout
        )
        self.decoder = TransformerDecoder(
            tgt_vocab_size, d_model, n_heads, n_decoder_layers, d_ff, max_len, dropout
        )
        # It's good practice to initialize weights, e.g., Xavier uniform.
        self._initialize_weights()

    def _initialize_weights(self):
        for p in self.parameters():
            if p.dim() > 1:         #only the weight tensor not bias
                nn.init.xavier_uniform_(p)

    def create_padding_mask(self, seq: torch.Tensor, pad_idx: int) -> torch.Tensor:
        """
        Creates a mask to hide padding tokens.
        Args:
            seq: Input sequence tensor, shape [batch_size, seq_len]
            pad_idx: Index of the padding token.
        Returns:
            mask: Boolean tensor, shape [batch_size, 1, 1, seq_len].
                  True for non-padding, False for padding.
                  Ready for broadcasting in MultiHeadAttention.
        """
        # seq != pad_idx results in a boolean tensor: True where not pad, False where pad.
        # .unsqueeze(1).unsqueeze(2) adds dimensions for broadcasting with attention scores
        # [batch_size, seq_len] -> [batch_size, 1, 1, seq_len]
        return (seq != pad_idx).unsqueeze(1).unsqueeze(2)

    def create_look_ahead_mask(self, size: int, device: torch.device) -> torch.Tensor:
        """
        Creates a look-ahead mask to prevent attending to future tokens in self-attention.
        Args:
            size: Sequence length of the target.
            device: The device to create the mask on.
        Returns:
            mask: Boolean tensor, shape [1, 1, size, size].
                  Lower triangle is True, upper is False.
        """
        # torch.ones creates a matrix of ones
        # torch.tril gets the lower triangular part
        mask = torch.tril(torch.ones(size, size, device=device)).bool() # [size, size]
        # .unsqueeze(0).unsqueeze(0) adds dimensions for broadcasting
        return mask.unsqueeze(0).unsqueeze(0) # [1, 1, size, size]

    def forward(self, src: torch.Tensor, tgt: torch.Tensor,
                src_pad_idx: int, tgt_pad_idx: int) -> torch.Tensor:
        """
        Forward pass for the Transformer model.
        Args:
            src: Source sequence tensor (token indices), shape [batch_size, src_len]
            tgt: Target sequence tensor (token indices), shape [batch_size, tgt_len]
            src_pad_idx: Padding index for the source vocabulary.
            tgt_pad_idx: Padding index for the target vocabulary.
        Returns:
            Output logits tensor from the decoder, shape [batch_size, tgt_len, tgt_vocab_size]
        """
        # 1. Create masks
        # Source padding mask (for encoder self-attention and decoder cross-attention)
        src_mask = self.create_padding_mask(src, src_pad_idx)  # [B, 1, 1, L_src]

        # Target padding mask (for decoder self-attention)
        tgt_padding_mask = self.create_padding_mask(tgt, tgt_pad_idx)  # [B, 1, 1, L_tgt]

        # Target look-ahead mask (for decoder self-attention to prevent seeing future tokens)
        look_ahead_mask = self.create_look_ahead_mask(tgt.size(1), tgt.device)  # [1, 1, L_tgt, L_tgt]

        # Combine target padding mask with look-ahead mask for decoder self-attention
        # Both should be True for a position to be attended.
        # tgt_padding_mask broadcasted: [B, 1, 1, L_tgt] -> [B, 1, L_tgt, L_tgt] (conceptually for the &)
        # look_ahead_mask: [1, 1, L_tgt, L_tgt]
        # Resulting tgt_mask: [B, 1, L_tgt, L_tgt]
        tgt_mask = tgt_padding_mask & look_ahead_mask

        # 2. Encoder pass
        encoder_output = self.encoder(src, src_mask) # encoder_output: [B, L_src, D_model]

        # 3. Decoder pass
        # src_mask for cross-attention is the same src_padding_mask.
        # tgt_mask is the combined mask for self-attention in decoder.
        decoder_output = self.decoder(tgt, encoder_output, src_mask, tgt_mask)

        return decoder_output


In [43]:

# --- Dataset and DataLoader ---

class TranslationDataset(Dataset):
    """
    Dataset for translation tasks.
    Assumes src_data and tgt_data are Pandas Series where each element is a list of token indices.
    """
    def __init__(self, src_data: pd.Series, tgt_data: pd.Series):
        self.src_data = src_data
        self.tgt_data = tgt_data

    def __len__(self) -> int:
        return len(self.src_data)

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        # .iloc[idx] retrieves the list of token indices from the Pandas Series
        src_tensor = torch.tensor(self.src_data.iloc[idx], dtype=torch.long)
        tgt_tensor = torch.tensor(self.tgt_data.iloc[idx], dtype=torch.long)
        return src_tensor, tgt_tensor


In [44]:

# --- Training and Evaluation Functions ---
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

# Assuming 'Transformer' is a defined class and imported

# --- Training and Evaluation Functions ---

def train_epoch(model: Transformer, dataloader: DataLoader, optimizer: torch.optim.Optimizer,
                criterion: nn.Module, device: torch.device, clip: float = 1.0,
                src_pad_idx: int = 0, tgt_pad_idx: int = 0, current_epoch: int = 0, total_epochs: int = 0) -> float:
    """Trains the model for one epoch."""
    model.train()  # Set model to training mode
    total_loss = 0
    num_batches = len(dataloader)

    for batch_idx, (src, tgt) in enumerate(dataloader):
        src, tgt = src.to(device), tgt.to(device) # src/tgt: [batch_size, seq_len]

        # Prepare target sequences for teacher forcing:
        # Decoder input: <sos> ... token_N (remove <eos>)
        tgt_input = tgt[:, :-1]
        # Target output: token_1 ... <eos> (remove <sos>)
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad()

        # Forward pass
        # The model's forward method handles mask creation internally if pad_idx is passed,
        # or we can pass them explicitly. Here, we pass pad_idx values.
        output_logits = model(src, tgt_input, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx)
        # output_logits shape: [batch_size, tgt_len-1, tgt_vocab_size]

        # Calculate loss
        # Reshape for CrossEntropyLoss:
        # output_logits: [(batch_size * (tgt_len-1)), tgt_vocab_size]
        # tgt_output: [batch_size * (tgt_len-1)]
        loss = criterion(output_logits.reshape(-1, output_logits.size(-1)), tgt_output.reshape(-1))

        loss.backward() # Compute gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) # Gradient clipping
        optimizer.step() # Update weights

        total_loss += loss.item()

        if (batch_idx + 1) % 100 == 0 or batch_idx == num_batches -1:
            print(f'  Epoch [{current_epoch+1}/{total_epochs}], Batch [{batch_idx+1}/{num_batches}], Loss: {loss.item():.4f}')

    return total_loss / num_batches

def evaluate_epoch(model: Transformer, dataloader: DataLoader, criterion: nn.Module,
                   device: torch.device, src_pad_idx: int = 0, tgt_pad_idx: int = 0) -> float:
    """Evaluates the model for one epoch."""
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    num_batches = len(dataloader)

    with torch.no_grad(): # Disable gradient calculations
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            output_logits = model(src, tgt_input, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx)
            loss = criterion(output_logits.reshape(-1, output_logits.size(-1)), tgt_output.reshape(-1))
            total_loss += loss.item()

    return total_loss / num_batches
    model.train()  # Set model to training mode
    total_loss = 0
    num_batches = len(dataloader)

    for batch_idx, (src, tgt) in enumerate(dataloader):
        src, tgt = src.to(device), tgt.to(device) # src/tgt: [batch_size, seq_len]

        # Prepare target sequences for teacher forcing:
        # Decoder input: <sos> ... token_N (remove <eos>)
        tgt_input = tgt[:, :-1]
        # Target output: token_1 ... <eos> (remove <sos>)
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad()

        # Forward pass
        # The model's forward method handles mask creation internally if pad_idx is passed,
        # or we can pass them explicitly. Here, we pass pad_idx values.
        output_logits = model(src, tgt_input, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx)
        # output_logits shape: [batch_size, tgt_len-1, tgt_vocab_size]

        # Calculate loss
        # Reshape for CrossEntropyLoss:
        # output_logits: [(batch_size * (tgt_len-1)), tgt_vocab_size]
        # tgt_output: [batch_size * (tgt_len-1)]
        loss = criterion(output_logits.reshape(-1, output_logits.size(-1)), tgt_output.reshape(-1))

        loss.backward() # Compute gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) # Gradient clipping
        optimizer.step() # Update weights

        total_loss += loss.item()

        if (batch_idx + 1) % 100 == 0 or batch_idx == num_batches -1:
            print(f'  Epoch [{current_epoch+1}/{total_epochs}], Batch [{batch_idx+1}/{num_batches}], Loss: {loss.item():.4f}')

    return total_loss / num_batches


In [48]:
# --- Training and Evaluation Functions ---
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import time # Import time for timing epochs

# Assuming 'Transformer' is a defined class and imported

# --- Training and Evaluation Functions ---

def train_epoch(model: Transformer, dataloader: DataLoader, optimizer: torch.optim.Optimizer,
                criterion: nn.Module, device: torch.device, clip: float = 1.0,
                src_pad_idx: int = 0, tgt_pad_idx: int = 0, current_epoch: int = 0, total_epochs: int = 0) -> float:
    """Trains the model for one epoch."""
    model.train()  # Set model to training mode
    total_loss = 0
    num_batches = len(dataloader)

    for batch_idx, (src, tgt) in enumerate(dataloader):
        src, tgt = src.to(device), tgt.to(device) # src/tgt: [batch_size, seq_len]

        # Prepare target sequences for teacher forcing:
        # Decoder input: <sos> ... token_N (remove <eos>)
        tgt_input = tgt[:, :-1]
        # Target output: token_1 ... <eos> (remove <sos>)
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad()

        # Forward pass
        # The model's forward method handles mask creation internally if pad_idx is passed,
        # or we can pass them explicitly. Here, we pass pad_idx values.
        output_logits = model(src, tgt_input, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx)
        # output_logits shape: [batch_size, tgt_len-1, tgt_vocab_size]

        # Calculate loss
        # Reshape for CrossEntropyLoss:
        # output_logits: [(batch_size * (tgt_len-1)), tgt_vocab_size]
        # tgt_output: [batch_size * (tgt_len-1)]
        loss = criterion(output_logits.reshape(-1, output_logits.size(-1)), tgt_output.reshape(-1))

        loss.backward() # Compute gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) # Gradient clipping
        optimizer.step() # Update weights

        total_loss += loss.item()

        # Added a print statement for progress feedback
        if (batch_idx + 1) % 100 == 0 or batch_idx == num_batches -1:
            print(f'  Epoch [{current_epoch+1}/{total_epochs}], Batch [{batch_idx+1}/{num_batches}], Loss: {loss.item():.4f}')


    return total_loss / num_batches

def evaluate_epoch(model: Transformer, dataloader: DataLoader, criterion: nn.Module,
                   device: torch.device, src_pad_idx: int = 0, tgt_pad_idx: int = 0) -> float:
    """Evaluates the model for one epoch."""
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    num_batches = len(dataloader)

    with torch.no_grad(): # Disable gradient calculations
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            output_logits = model(src, tgt_input, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx)
            loss = criterion(output_logits.reshape(-1, output_logits.size(-1)), tgt_output.reshape(-1))
            total_loss += loss.item()

    return total_loss / num_batches


def train_transformer_model(model: Transformer, train_dataloader: DataLoader, test_dataloader: DataLoader,
                           num_epochs: int, device: torch.device, learning_rate: float, warmup_steps: int,
                           src_pad_idx: int, tgt_pad_idx: int, clip: float = 1.0, model_save_path: str = None):

    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-9)

    # Ignore padding index in loss calculation
    criterion = nn.CrossEntropyLoss(ignore_index=tgt_pad_idx)

    best_valid_loss = float('inf')

    for epoch in range(num_epochs):
        start_time = time.time()

        # Train
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion, device,
                                 clip, src_pad_idx, tgt_pad_idx, epoch, num_epochs)

        # Evaluate
        valid_loss = evaluate_epoch(model, test_dataloader, criterion, device,
                                    src_pad_idx, tgt_pad_idx)

        end_time = time.time()
        epoch_mins, epoch_secs = divmod(end_time - start_time, 60)

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.0f}s')
        print(f'\tTrain Loss: {train_loss:.3f}')
        print(f'\t Valid Loss: {valid_loss:.3f}')

        # Save the best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            if model_save_path:
                torch.save(model.state_dict(), model_save_path)
                print(f'Saved best model to {model_save_path}')

    print("Training complete.")
    return model # Optionally return the trained model

In [49]:

def evaluate_epoch(model: Transformer, dataloader: DataLoader, criterion: nn.Module,
                   device: torch.device, src_pad_idx: int, tgt_pad_idx: int) -> float:
    """Evaluates the model for one epoch."""
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    num_batches = len(dataloader)

    with torch.no_grad(): # Disable gradient calculations
        for src, tgt in dataloader:
            src, tgt = src.to(device), tgt.to(device)

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            output_logits = model(src, tgt_input, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx)
            loss = criterion(output_logits.reshape(-1, output_logits.size(-1)), tgt_output.reshape(-1))
            total_loss += loss.item()

    return total_loss / num_batches
    model.train()  # Set model to training mode
    total_loss = 0
    num_batches = len(dataloader)

    for batch_idx, (src, tgt) in enumerate(dataloader):
        src, tgt = src.to(device), tgt.to(device) # src/tgt: [batch_size, seq_len]

        # Prepare target sequences for teacher forcing:
        # Decoder input: <sos> ... token_N (remove <eos>)
        tgt_input = tgt[:, :-1]
        # Target output: token_1 ... <eos> (remove <sos>)
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad()

        # Forward pass
        # The model's forward method handles mask creation internally if pad_idx is passed,
        # or we can pass them explicitly. Here, we pass pad_idx values.
        output_logits = model(src, tgt_input, src_pad_idx=src_pad_idx, tgt_pad_idx=tgt_pad_idx)
        # output_logits shape: [batch_size, tgt_len-1, tgt_vocab_size]

        # Calculate loss
        # Reshape for CrossEntropyLoss:
        # output_logits: [(batch_size * (tgt_len-1)), tgt_vocab_size]
        # tgt_output: [batch_size * (tgt_len-1)]
        loss = criterion(output_logits.reshape(-1, output_logits.size(-1)), tgt_output.reshape(-1))

        loss.backward() # Compute gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) # Gradient clipping
        optimizer.step() # Update weights

        total_loss += loss.item()

        if (batch_idx + 1) % 100 == 0 or batch_idx == num_batches -1:
            print(f'  Epoch [{current_epoch+1}/{total_epochs}], Batch [{batch_idx+1}/{num_batches}], Loss: {loss.item():.4f}')

    return total_loss / num_batches


# MODEL TRAINING

In [ ]:
if __name__ == "__main__":

    SRC_VOCAB_SIZE = len(engword) # Use the size of the English vocabulary
    TGT_VOCAB_SIZE = len(spaword) # Use the size of the Spanish vocabulary

    max_eng_len = 0
    max_spa_len = 0
    for eng_batch, spa_batch in train_dataloader:
        max_eng_len = max(max_eng_len, eng_batch.size(1))
        max_spa_len = max(max_spa_len, spa_batch.size(1))

    # MAX_SEQ_LEN should be at least the maximum length found in the data.
    # The decoder input tgt_input[:, :-1] will have length max_spa_len - 1,
    # but the positional encoding is applied to the original embedded_tgt which has length max_spa_len.
    MAX_SEQ_LEN = max(max_eng_len, max_spa_len)
    print(f"Determined MAX_SEQ_LEN from data: {MAX_SEQ_LEN}")


    # Define special token strings
    PAD_TOKEN = '<pad>'
    SOS_TOKEN = '<sos>'
    EOS_TOKEN = '<eos>'

    # --- 1. Model Hyperparameters ---
    D_MODEL = 256 # Reduced for faster training (Original paper: 512)
    N_HEADS = 8
    N_ENCODER_LAYERS = 3 # Reduced (Original paper: 6)
    N_DECODER_LAYERS = 3 # Reduced (Original paper: 6)
    D_FF = 512      # Reduced (Original paper: 2048)
    DROPOUT = 0.1

    # --- 2. Training Parameters ---
    BATCH_SIZE = 64
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.0005 # Adjusted common starting LR
    WARMUP_STEPS = 0 # For simplified LR scheduler. For Transformer paper schedule, use e.g. 4000
    MODEL_SAVE_PATH = 'best_transformer_model.pt'

    # --- 3. Device Configuration ---
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # --- 4. Initialize Model ---
    model = Transformer(
        # Pass the integer size of the vocabulary, not the array itself
        src_vocab_size=SRC_VOCAB_SIZE,
        tgt_vocab_size=TGT_VOCAB_SIZE,
        d_model=D_MODEL,
        n_heads=N_HEADS,
        n_encoder_layers=N_ENCODER_LAYERS,
        n_decoder_layers=N_DECODER_LAYERS,
        d_ff=D_FF,
        max_len=MAX_SEQ_LEN, # Use the determined MAX_SEQ_LEN
        dropout=DROPOUT
    ).to(device)

    print(f'Transformer Model initialized with {sum(p.numel() for p in model.parameters() if p.requires_grad):,} trainable parameters.')

    # --- 5. (Optional) Check a batch shape after loading dataloader ---
    # This confirms the actual tensor shapes being fed into the model
    # for src_batch, tgt_batch in train_dataloader:
    #     print(f"Example batch shapes from train_dataloader: src={src_batch.shape}, tgt={tgt_batch.shape}")
    #     break # Only check the first batch


    # --- 6. Train the Model ---
    trained_model = train_transformer_model(
        model, train_dataloader, test_dataloader, NUM_EPOCHS, device,
        LEARNING_RATE, WARMUP_STEPS,
        src_pad_idx=eng_tok_stoi['<pad>'],  # Pass English pad index
        tgt_pad_idx=spa_tok_stoi['<pad>'],  # Pass Spanish pad index (for loss and masks)
        model_save_path=MODEL_SAVE_PATH
    )


    model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
    print(f"\nLoaded best model from {MODEL_SAVE_PATH} for translation.")

Determined MAX_SEQ_LEN from data: 40
Using device: cuda
Transformer Model initialized with 16,468,746 trainable parameters.
  Epoch [1/10], Batch [100/744], Loss: 5.7439
  Epoch [1/10], Batch [200/744], Loss: 5.4637
  Epoch [1/10], Batch [300/744], Loss: 5.2471
  Epoch [1/10], Batch [400/744], Loss: 4.8346
  Epoch [1/10], Batch [500/744], Loss: 4.6414
  Epoch [1/10], Batch [600/744], Loss: 4.3983
  Epoch [1/10], Batch [700/744], Loss: 4.3128
  Epoch [1/10], Batch [744/744], Loss: 4.2308
Epoch: 01 | Time: 0.0m 49s
	Train Loss: 5.119
	 Valid Loss: 4.148
Saved best model to best_transformer_model.pt
  Epoch [2/10], Batch [100/744], Loss: 4.0693
  Epoch [2/10], Batch [200/744], Loss: 3.9534
  Epoch [2/10], Batch [300/744], Loss: 3.9136
  Epoch [2/10], Batch [400/744], Loss: 3.3747
  Epoch [2/10], Batch [500/744], Loss: 3.5134
  Epoch [2/10], Batch [600/744], Loss: 3.2371
  Epoch [2/10], Batch [700/744], Loss: 3.2565
  Epoch [2/10], Batch [744/744], Loss: 3.4447
Epoch: 02 | Time: 0.0m 48s
	

# TRANSLATION

In [52]:

def translate_sentence(sentence: str, model: Transformer, spacy_en, spacy_es,
                       eng_stoi: dict, spa_itos: dict, englen: int, spalen: int,
                       device: torch.device, src_pad_idx: int, sos_idx: int, eos_idx: int):

    model.eval() # Set model to evaluation mode

    # 1. Preprocess the input sentence
    # Tokenize the English sentence
    tokens = [tok.text.lower() for tok in spacy_en.tokenizer(sentence)]
    # Ensure sentence is not empty after tokenization
    if not tokens:
        return "Error: Input sentence is empty or could not be tokenized."

    # Convert tokens to indices
    # Handle unknown tokens (tokens not in eng_stoi) - replace with a special <unk> token if available,
    # or handle gracefully. Assuming no <unk> for simplicity, skip unknown tokens or use a default.
    # A better approach would include an <unk> token in the vocabulary.
    # For this example, let's just skip unknown tokens, which might lead to errors if a token is critical.
    # Let's add a check and use a placeholder or skip. A more robust solution needs an <unk> token.
    src_indices = [eng_stoi.get(token, src_pad_idx) for token in tokens] # Using PAD_TOKEN for simplicity if unknown

    # Add padding to match the max length used during training
    if len(src_indices) < englen:
        src_indices += [src_pad_idx] * (englen - len(src_indices))
    else:
        # Truncate if longer than max length used in training (less common in translation inference)
        src_indices = src_indices[:englen]
        print(f"Warning: Input sentence truncated to {englen} tokens.")


    # Convert to tensor and add batch dimension (batch_size=1)
    src_tensor = torch.tensor(src_indices, dtype=torch.long).unsqueeze(0).to(device) # [1, englen]

    # 2. Encode the source sequence
    with torch.no_grad():
        # Create source padding mask for the encoder
        src_mask = model.create_padding_mask(src_tensor, src_pad_idx) # [1, 1, 1, englen]
        encoder_output = model.encoder(src_tensor, src_mask) # [1, englen, D_MODEL]

    # 3. Decode the target sequence word by word (autonomously)
    # Start with the <sos> token
    # Initially, the target sequence for the decoder is just the <sos> token
    tgt_indices = [sos_idx]
    # Convert to tensor and add batch dimension
    tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long).unsqueeze(0).to(device) # [1, 1]

    # Iterate until <eos> is predicted or maximum sequence length is reached
    for _ in range(spalen - 1): # spalen is the max length including <sos> and <eos>
        with torch.no_grad():
            # Create combined target mask for the current tgt_tensor
            # Need padding mask for the current tgt_tensor (only <sos> initially, then grows)
            # Need look-ahead mask for the current length of tgt_tensor
            tgt_padding_mask = model.create_padding_mask(tgt_tensor, src_pad_idx) # Use src_pad_idx if pad_idx is shared [1, 1, 1, current_tgt_len]
                                                                               # Or use the correct target pad idx if different. Assuming shared for <pad> token.
            look_ahead_mask = model.create_look_ahead_mask(tgt_tensor.size(1), device) # [1, 1, current_tgt_len, current_tgt_len]
            combined_tgt_mask = tgt_padding_mask & look_ahead_mask

            # Forward pass through the decoder
            # encoder_output is reused at each step
            # src_mask is reused at each step
            # tgt_tensor grows by one token at each step
            output_logits = model.decoder(tgt_tensor, encoder_output, src_mask, combined_tgt_mask)
            # output_logits shape: [1, current_tgt_len, TGT_VOCAB_SIZE]

            # Get the prediction for the *last* token in the sequence
            prediction = output_logits[:, -1, :] # [1, TGT_VOCAB_SIZE]
            predicted_token_index = prediction.argmax(dim=-1).item() # Get the index of the most probable token

        # Append the predicted token index to the target sequence
        tgt_indices.append(predicted_token_index)

        # Convert the updated target indices to a tensor for the next step
        tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long).unsqueeze(0).to(device) # [1, current_tgt_len + 1]

        # If the predicted token is <eos>, stop decoding
        if predicted_token_index == eos_idx:
            break

        # If the target sequence reaches the max length, stop decoding
        if len(tgt_indices) >= spalen:
            print(f"Warning: Decoding stopped at max length {spalen}.")
            break


    # 4. Convert predicted indices back to tokens
    # Exclude the <sos> token from the output translation
    translated_tokens = [spa_itos.get(idx, '<unk>') for idx in tgt_indices[1:]] # Use <unk> if index not found

    # Remove <eos> token if it was the last predicted token
    if translated_tokens and translated_tokens[-1] == EOS_TOKEN:
        translated_tokens = translated_tokens[:-1]

    # Join tokens to form the translated sentence
    # Decide how to join (e.g., with space). For Spanish, usually space is appropriate.
    translated_sentence = " ".join(translated_tokens)

    return translated_sentence

# --- Example Usage (assuming the rest of your code has run successfully) ---

# Ensure the model is on the correct device after loading
model.to(device)

# Assuming these dictionaries and lengths are available from previous steps
# eng_tok_stoi, spa_tok_itos, englen, spalen, device
# You also need the indices for special tokens
try:
    src_pad_idx_val = eng_tok_stoi[PAD_TOKEN] # Assuming same pad token index in src and tgt
    sos_idx_val = spa_tok_stoi[SOS_TOKEN]
    eos_idx_val = spa_tok_stoi[EOS_TOKEN]
    # If <pad> has a different index in Spanish vocab, use that for tgt_mask in inference:
    # tgt_pad_idx_val = spa_tok_stoi[PAD_TOKEN]

except KeyError as e:
    print(f"Error: Special token index not found in vocabulary: {e}")
    print("Please ensure '<pad>', '<sos>', and '<eos>' are in your vocabularies.")
    # Handle error, e.g., exit or use default values if applicable
    exit() # Exit if required indices are missing


# Example sentences to translate
sentences_to_translate = [
    "hello world",
    "how are you",
    "i am learning spanish",
    "thank you very much",
    "this is a test sentence",
    "i like apples and oranges"
]

print("\n--- Translation Examples ---")
for sentence in sentences_to_translate:
    translated = translate_sentence(
        sentence=sentence,
        model=model,
        spacy_en=spacy_en, # Assuming spacy_en is loaded
        spacy_es=spacy_es, # Assuming spacy_es is loaded
        eng_stoi=eng_tok_stoi,
        spa_itos=spa_tok_itos,
        englen=englen,
        spalen=spalen, # Use the length calculated *after* adding sos/eos/pad
        device=device,
        src_pad_idx=src_pad_idx_val, # Use the English pad index
        sos_idx=sos_idx_val,
        eos_idx=eos_idx_val
    )
    print(f"English: \"{sentence}\"")
    print(f"Spanish: \"{translated}\"")
    print("-" * 20)



--- Translation Examples ---
English: "hello world"
Spanish: "dile hola el mundo"
--------------------
English: "how are you"
Spanish: "¿ cómo te estás tú !"
--------------------
English: "i am learning spanish"
Spanish: "estoy aprendiendo español"
--------------------
English: "thank you very much"
Spanish: "gracias mucho mucho"
--------------------
English: "this is a test sentence"
Spanish: "esta frase es una frase"
--------------------
English: "i like apples and oranges"
Spanish: "me gustan las manzanas y las naranjas"
--------------------


# Load the model

In [53]:

model.to(device)
try:
  # Use the actual padding index from the English vocabulary
  src_pad_idx_val = eng_tok_stoi['<pad>']
  sos_idx_val = spa_tok_stoi['<sos>']
  eos_idx_val = spa_tok_stoi['<eos>']

except KeyError as e:
    print(f"Error: Special token index not found in vocabulary: {e}")
    print("Please ensure '<pad>', '<sos>', and '<eos>' are in your vocabularies.")
    # Handle error, e.g., exit or use default values if applicable
    exit() # Exit if required indices are missing


# Sentence to translate
sentence_to_translate = 'i love spanish'

print(f"\n--- Translating: \"{sentence_to_translate}\" ---")

translated = translate_sentence(
    sentence=sentence_to_translate,
    model=model,
    spacy_en=spacy_en, # Assuming spacy_en is loaded
    spacy_es=spacy_es, # Assuming spacy_es is loaded
    eng_stoi=eng_tok_stoi,
    spa_itos=spa_tok_itos,
    englen=englen,
    spalen=spalen,
    device=device,
    src_pad_idx=src_pad_idx_val, # Use the English pad index
    sos_idx=sos_idx_val,
    eos_idx=eos_idx_val
)

print(f"English: \"{sentence_to_translate}\"")
print(f"Spanish: \"{translated}\"")
print("-" * 20)


--- Translating: "i love spanish" ---
English: "i love spanish"
Spanish: "me encanta español"
--------------------


**Here we can see the translation of the model is wayyy better from the translation from encoder deocoder architecture under same epoch . It's almost close to real translation with just 10 epoch**